In [1]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

import torch
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

from data_utility import *
from data_utils import *
from loss import *
from train import *
from deeplab_model.deeplab import *
from dense_vnet.DenseVNet import DenseVNet
from sync_batchnorm import convert_model
import datetime

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
USE_GPU = True
NUM_WORKERS = 12
BATCH_SIZE = 2 

dtype = torch.float32 
# define dtype, float is space efficient than double

if USE_GPU and torch.cuda.is_available():
    
    device = torch.device("cuda:0")
    
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True
    # magic flag that accelerate
    
    print('using GPU for training')
else:
    device = torch.device('cpu')
    print('using CPU for training')

using GPU for training


In [3]:
train_dataset = get_full_resolution_dataset(data_type = 'nii_train', 
                transform=transforms.Compose([
                random_affine(90, 15),
                random_filp(0.5)]))
# do data augumentation on train dataset

validation_dataset = get_full_resolution_dataset(data_type = 'nii_test', 
                transform=None)
# no data augumentation on validation dataset

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                    num_workers=NUM_WORKERS)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True,
                    num_workers=NUM_WORKERS) # drop_last
# loaders come with auto batch division and multi-thread acceleration

In [4]:
from vnet import VNet
from bv_refinement_network.RefinementModel import RefinementModel_ELU
from refinenet import refine_net

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    
#checkpoint_refine = torch.load('../refine_bv_resize_save/2019-08-22 00:10:23.854113 epoch: 41.pth')
    
refine_model = refine_net(num_classes=1, in_channels=1)
#refine_model = nn.DataParallel(refine_model)
#refine_model = convert_model(refine_model)

#refine_model.load_state_dict(checkpoint_refine['state_dict_1'])

refine_model = refine_model.to(device, dtype)

optimizer = optim.Adam(refine_model.parameters(), lr=1e-3)
#optimizer.load_state_dict(checkpoint_refine['optimizer'])

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)
#scheduler.load_state_dict(checkpoint_refine['scheduler'])

deeplab = DeepLab(output_stride=16)
deeplab = nn.DataParallel(deeplab)
deeplab = convert_model(deeplab)

checkpoint = torch.load('../deeplab_dilated_save/2019-08-10 09:28:43.844872 epoch: 1160.pth') # best one

deeplab.load_state_dict(checkpoint['state_dict_1'])
deeplab = deeplab.to(device, dtype)

epoch = 0
#epoch = 0
print(epoch)

Let's use 2 GPUs!
0


In [ ]:
def get_bboxes(image, label, output, batchsize, box_size):
    image_final = torch.zeros((batchsize, 1, box_size, box_size, box_size)).to(device, dtype)
    label_final = torch.zeros((batchsize, 1, box_size, box_size, box_size)).to(device, dtype)
    output_final = torch.zeros((batchsize, 1, box_size, box_size, box_size)).to(device, dtype)
    half_size = int(box_size/2)
    image_size_x = int(image.shape[-3])
    image_size_y = int(image.shape[-2])
    image_size_z = int(image.shape[-1])
    for b in range(batchsize):
        out = output[b]
        x,y,z = loadbvcenter(binarize_output(out))
        x, y, z = np.clip([x, y, z], a_min=half_size, a_max=181)
        x1 = max(x-half_size, 0)
        x2 = min(x+half_size, image_size_x)
        y1 = max(y-half_size, 0)
        y2 = min(y+half_size, image_size_y)
        z1 = max(z-half_size, 0)
        z2 = min(z+half_size, image_size_z)
        image_final[b] = image[b, :, x1:x2, y1:y2, z1:z2]
        label_final[b] = label[b, :, x1:x2, y1:y2, z1:z2]
        output_final[b] = output[b, :, x1:x2, y1:y2, z1:z2]
    return image_final, label_final, output_final

In [ ]:
epochs = 5000

record = open('train_bv_refine_resize_no_concat_2.txt','a+')

logger = {'train':[], 'validation_1': []}

min_val = 1

for e in tqdm(range(epoch + 1, epochs)):
# iter over epoches
    epoch_loss = 0
        
    for t, batch in enumerate(train_loader):
    # iter over the train mini batches
        train_losses=[]
        for minibatch in range(BATCH_SIZE):
            
            refine_model.train()
            deeplab.eval()
            # Set the model flag to train
            # 1. enable dropout
            # 2. batchnorm behave differently in train and test
            #print(batch['image1_data'])
            image_1 = batch['image1_data'][minibatch].to(device=device, dtype=dtype)
            image_1 = image_1.view(1,1,256,256,256)

            label_1 = batch['image1_label'][minibatch].to(device=device, dtype=dtype)
            label_1 = label_1.view(1,3,256,256,256)

            bv_label = label_1[:, 2, :, :, :]
            bv_label = bv_label.view(1,1,256,256,256)

            original_res = [a[minibatch].item() for a in batch['original_resolution']]

            image_1_resize = F.interpolate(image_1, size=original_res, mode='trilinear', align_corners=True)
            image_1_resize = image_1_resize.view(1,1,original_res[0], original_res[1], original_res[2])

            bv_label_resize = F.interpolate(bv_label, size=original_res, mode='trilinear', align_corners=True)

            # Get coarse output from deeplab model from 256 resolution input
            out_coarse = deeplab(image_1)
            out_coarse = out_coarse.view(1,3,256,256,256)
            
            bv_coarse = out_coarse[:, 2, :, :, :]
            bv_coarse = bv_coarse.view(1,1,256,256,256)
            
            del out_coarse
            
            bv_coarse_resize = F.interpolate(bv_coarse, size=original_res, mode='trilinear', align_corners=True)

            del bv_coarse

            box_size = 192
            half_size = int(box_size / 2)

            image_size_x = int(image_1_resize.shape[-3])
            image_size_y = int(image_1_resize.shape[-2])
            image_size_z = int(image_1_resize.shape[-1])

            x,y,z = loadbvcenter(binarize_output(bv_coarse_resize).view([1] + original_res))
            x, y, z = np.clip([x, y, z], a_min=box_size-half_size, a_max=box_size+half_size)
            x1 = max(x-half_size, 0)
            x2 = min(x+half_size, image_size_x)
            y1 = max(y-half_size, 0)
            y2 = min(y+half_size, image_size_y)
            z1 = max(z-half_size, 0)
            z2 = min(z+half_size, image_size_z)

            bbox_bv = bv_coarse_resize.view(original_res)[x1:x2, y1:y2, z1:z2]
            bbox_bv = reshape_image(bbox_bv.squeeze(), box_size, box_size, box_size).to(device, dtype)
            bbox_bv = bbox_bv.view(1,1,box_size,box_size,box_size)

            del bv_coarse_resize

            bbox_bv_label = bv_label_resize.view(original_res)[x1:x2, y1:y2, z1:z2]
            bbox_bv_label = reshape_image(bbox_bv_label.squeeze(), box_size, box_size, box_size).to(device, dtype)
            bbox_bv_label = bbox_bv_label.view(1,1,box_size,box_size,box_size)

            del bv_label_resize

            bbox_image = image_1_resize[:, :, x1:x2, y1:y2, z1:z2]
            bbox_image = reshape_image(bbox_image.squeeze(), box_size, box_size, box_size).to(device, dtype)
            bbox_image = bbox_image.view(1, 1, box_size, box_size, box_size)

            del image_1_resize

            bbox_image_2 = F.interpolate(bbox_image, scale_factor=1/2, mode='trilinear', align_corners=True)
            bbox_image_4 = F.interpolate(bbox_image, scale_factor=1/4, mode='trilinear', align_corners=True)

            refine_out = refine_model(bbox_image, bbox_image_2, bbox_image_4)

            del bbox_image
            del bbox_image_2
            del bbox_image_4

            loss = dice_loss(refine_out, bbox_bv_label)

            del refine_out
            del bbox_bv_label

            torch.cuda.empty_cache()
            
            print(loss)
            train_losses.append(loss)
        
        loss = sum(train_losses) / BATCH_SIZE
        train_losses=[]
        epoch_loss += loss.item()
        # record minibatch loss to epoch loss
        
        optimizer.zero_grad()
        # set the model parameter gradient to zero
        
        loss.backward()
        # calculate the gradient wrt loss
        optimizer.step()
        #scheduler.step(loss_1)
        # take a gradient descent step
        
    outstr = 'Epoch {0} finished ! Training Loss: {1:.4f}'.format(e, epoch_loss/(t+1)) + '\n'
    
    logger['train'].append(epoch_loss/(t+1))
    
    print(outstr)
    record.write(outstr)
    record.flush()

    if e%1 == 0:
    # do validation every 5 epoches
        deeplab.eval()
        refine_model.eval()
        # set model flag to eval
        # 1. disable dropout
        # 2. batchnorm behave differs

        with torch.no_grad():
        # stop taking gradient
        
            #valloss_4 = 0
            #valloss_2 = 0
            valloss_1 = 0
            
            for v, vbatch in enumerate(validation_loader):
            # iter over validation mini batches
                val_losses = []
                for minibatch in range(BATCH_SIZE):
                    
                    image_1 = vbatch['image1_data'][minibatch].to(device=device, dtype=dtype)
                    image_1 = image_1.view(1,1,256,256,256)

                    label_1 = vbatch['image1_label'][minibatch].to(device=device, dtype=dtype)
                    label_1 = label_1.view(1,3,256,256,256)

                    bv_label = label_1[:, 2, :, :, :]
                    bv_label = bv_label.view(1,1,256,256,256)

                    original_res = [a[minibatch].item() for a in vbatch['original_resolution']]

                    image_1_resize = F.interpolate(image_1, size=original_res, mode='trilinear', align_corners=True)
                    image_1_resize = image_1_resize.view(1,1,original_res[0], original_res[1], original_res[2])

                    bv_label_resize = F.interpolate(bv_label, size=original_res, mode='trilinear', align_corners=True)

                    # Get coarse output from deeplab model from 256 resolution input
                    out_coarse = deeplab(image_1)
                    out_coarse = out_coarse.view(1,3,256,256,256)

                    bv_coarse = out_coarse[:, 2, :, :, :]
                    bv_coarse = bv_coarse.view(1,1,256,256,256)
                    
                    del out_coarse

                    bv_coarse_resize = F.interpolate(bv_coarse, size=original_res, mode='trilinear', align_corners=True)
                    
                    del bv_coarse
                    
                    box_size = 192
                    half_size = int(box_size / 2)

                    image_size_x = int(image_1_resize.shape[-3])
                    image_size_y = int(image_1_resize.shape[-2])
                    image_size_z = int(image_1_resize.shape[-1])

                    x,y,z = loadbvcenter(binarize_output(bv_coarse_resize).view([1] + original_res))
                    x, y, z = np.clip([x, y, z], a_min=box_size-half_size, a_max=box_size+half_size)
                    x1 = max(x-half_size, 0)
                    x2 = min(x+half_size, image_size_x)
                    y1 = max(y-half_size, 0)
                    y2 = min(y+half_size, image_size_y)
                    z1 = max(z-half_size, 0)
                    z2 = min(z+half_size, image_size_z)

                    bbox_bv = bv_coarse_resize.view(original_res)[x1:x2, y1:y2, z1:z2]
                    bbox_bv = reshape_image(bbox_bv.squeeze(), box_size, box_size, box_size).to(device, dtype)
                    bbox_bv = bbox_bv.view(1,1,box_size,box_size,box_size)
                    
                    del bv_coarse_resize
                    
                    bbox_bv_label = bv_label_resize.view(original_res)[x1:x2, y1:y2, z1:z2]
                    bbox_bv_label = reshape_image(bbox_bv_label.squeeze(), box_size, box_size, box_size).to(device, dtype)
                    bbox_bv_label = bbox_bv_label.view(1,1,box_size,box_size,box_size)

                    del bv_label_resize
                    
                    bbox_image = image_1_resize[:, :, x1:x2, y1:y2, z1:z2]
                    bbox_image = reshape_image(bbox_image.squeeze(), box_size, box_size, box_size).to(device, dtype)
                    bbox_image = bbox_image.view(1, 1, box_size, box_size, box_size)
                    
                    del image_1_resize
                    
                    bbox_image_2 = F.interpolate(bbox_image, scale_factor=1/2, mode='trilinear', align_corners=True)
                    bbox_image_4 = F.interpolate(bbox_image, scale_factor=1/4, mode='trilinear', align_corners=True)

                    refine_out = refine_model(bbox_image, bbox_image_2, bbox_image_4)
                    
                    del bbox_image
                    del bbox_image_2
                    del bbox_image_4
                    
                    loss = dice_loss(refine_out, bbox_bv_label)
                    
                    del refine_out
                    del bbox_bv_label
                    
                    torch.cuda.empty_cache()
                    
                    val_losses.append(loss)
                
                avg_loss = sum(val_losses) / BATCH_SIZE
                val_losses = []
                print(avg_loss)
            
                # calculate loss
                valloss_1 += avg_loss.item()
                
            
            avg_val_loss = (valloss_1 / (v+1))
            outstr = '------- 1st valloss={0:.4f}'\
                .format(avg_val_loss) + '\n'
            
            logger['validation_1'].append(avg_val_loss)
            #scheduler.step(avg_val_loss)
            
            if avg_val_loss < min_val:
                min_val = avg_val_loss
                save_1('refine_bv_resize_no_concat_2_save', refine_model, optimizer, logger, e, scheduler)
            elif e % 10 == 0:
                save_1('refine_bv_resize_no_concat_2_save', refine_model, optimizer, logger, e, scheduler)
            
            print(outstr)
            record.write(outstr)
            record.flush()
    


record.close()

  0%|          | 0/4999 [00:00<?, ?it/s]

tensor(0.9941, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9978, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9918, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9873, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9916, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9902, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9832, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9915, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9998, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9988, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9923, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9825, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(1., device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9885, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9794, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9855, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9883, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9860, device='cuda:0', gra

tensor(0.9828, device='cuda:0')
tensor(0.9858, device='cuda:0')
tensor(0.9788, device='cuda:0')
tensor(0.9725, device='cuda:0')
tensor(0.9826, device='cuda:0')


  0%|          | 1/4999 [06:26<535:54:49, 386.01s/it]

tensor(0.9893, device='cuda:0')
Checkpoint 1 saved !
------- 1st valloss=0.9849

tensor(0.9804, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(1., device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9975, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9834, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9706, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9733, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9736, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(1., device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9788, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9886, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(1., device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9858, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9834, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9915, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(1., device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9918, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9903, device='

In [ ]:
deeplab.eval()
refine_model.eval()

with torch.no_grad():

    val_loss = 0
    
    for v, vbatch in tqdm(enumerate(validation_loader)):
        # move data to device, convert dtype to desirable dtype
        val_losses = []
        for minibatch in range(BATCH_SIZE):
            image_1 = vbatch['image1_data'][minibatch].to(device=device, dtype=dtype)
            image_1 = image_1.view(1,1,256,256,256)

            label_1 = vbatch['image1_label'][minibatch].to(device=device, dtype=dtype)
            label_1 = label_1.view(1,3,256,256,256)

            bv_label = label_1[:, 2, :, :, :]
            bv_label = bv_label.view(1,1,256,256,256)

            original_res = [a[minibatch].item() for a in vbatch['original_resolution']]

            image_1_resize = F.interpolate(image_1, size=original_res, mode='trilinear', align_corners=True)
            image_1_resize = image_1_resize.view(1,1,original_res[0], original_res[1], original_res[2])

            bv_label_resize = F.interpolate(bv_label, size=original_res, mode='trilinear', align_corners=True)

            # Get coarse output from deeplab model from 256 resolution input
            out_coarse = deeplab(image_1)
            out_coarse = out_coarse.view(1,3,256,256,256)

            bv_coarse = out_coarse[:, 2, :, :, :]
            bv_coarse = bv_coarse.view(1,1,256,256,256)

            bv_coarse_resize = F.interpolate(bv_coarse, size=original_res, mode='trilinear', align_corners=True)
            
            box_size = 192
            half_size = int(box_size / 2)
            
            image_size_x = int(image_1_resize.shape[-3])
            image_size_y = int(image_1_resize.shape[-2])
            image_size_z = int(image_1_resize.shape[-1])
            
            x,y,z = loadbvcenter(binarize_output(bv_coarse_resize).view([1] + original_res))
            x, y, z = np.clip([x, y, z], a_min=box_size-half_size, a_max=box_size+half_size)
            x1 = max(x-half_size, 0)
            x2 = min(x+half_size, image_size_x)
            y1 = max(y-half_size, 0)
            y2 = min(y+half_size, image_size_y)
            z1 = max(z-half_size, 0)
            z2 = min(z+half_size, image_size_z)
            
            
            bbox_bv = bv_coarse_resize.view(original_res)[x1:x2, y1:y2, z1:z2]
            bbox_bv = reshape_image(bbox_bv.squeeze(), box_size, box_size, box_size).to(device, dtype)
            bbox_bv = bbox_bv.view(1,1,box_size,box_size,box_size)
            
            bbox_bv_label = bv_label_resize.view(original_res)[x1:x2, y1:y2, z1:z2]
            bbox_bv_label = reshape_image(bbox_bv_label.squeeze(), box_size, box_size, box_size).to(device, dtype)
            bbox_bv_label = bbox_bv_label.view(1,1,box_size,box_size,box_size)

            #bbox_image = get_bounding_box_image(image_1, (256,256,256)).to(device, dtype)
            bbox_image = image_1_resize[:, :, x1:x2, y1:y2, z1:z2]
            bbox_image = reshape_image(bbox_image.squeeze(), box_size, box_size, box_size).to(device, dtype)
            bbox_image = bbox_image.view(1, 1, box_size, box_size, box_size)
            
            #bbox_iamge, bbox_bv_label, bbox_bv = get_bboxes(image_1_resize, bv_label_resize, bv_coarse_resize, 1, 200)
            
            bbox_concat = torch.cat([bbox_bv, bbox_image], dim=1)
            bbox_concat_2 = F.interpolate(bbox_concat, scale_factor=1/2, mode='trilinear', align_corners=True)
            bbox_concat_4 = F.interpolate(bbox_concat, scale_factor=1/4, mode='trilinear', align_corners=True)

            refine_out = refine_model(bbox_concat, bbox_concat_2, bbox_concat_4)

            loss = dice_loss(refine_out, bbox_bv_label)
            val_losses.append(loss)
            
            if loss.item() > .04:
                show_image_slice(image_1)
                show_image_slice(bv_label_resize)
                show_image_slice(bv_coarse)
                show_image_slice(bbox_image)
                show_image_slice(bbox_bv_label)
                show_image_slice(bbox_bv)
                show_image_slice(refine_out)
        
        loss = sum(val_losses) / BATCH_SIZE
        print(loss.item())
        val_loss += loss.item()
        val_losses = []
        '''
        if loss.item() > .05:
            show_image_slice(image_1)
            show_image_slice(label_1)
            show_image_slice(output)
        '''

    outstr = 'bv loss = {0:.4f}'\
        .format(val_loss/(v+1)) + '\n'
    print(outstr)